In [7]:
import numpy as np
import pandas as pd
import re
import random
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
lemmatizer = WordNetLemmatizer()

import json
import sqlite3
from nltk import word_tokenize

In [8]:
def tokenize(string):
    string = str(string)
    string = string.replace("\'", "\"")  # ensures all string values wrapped by "" problem??
    quote_idxs = [idx for idx, char in enumerate(string) if char == '"']
    assert len(quote_idxs) % 2 == 0, "Unexpected quote"

    # keep string value as token
    vals = {}
    for i in range(len(quote_idxs)-1, -1, -2):
        qidx1 = quote_idxs[i-1]
        qidx2 = quote_idxs[i]
        val = string[qidx1: qidx2+1]
        key = "__val_{}_{}__".format(qidx1, qidx2)
        string = string[:qidx1] + key + string[qidx2+1:]
        vals[key] = val

    toks = [word.lower() for word in word_tokenize(string)]
    # replace with string value token
    for i in range(len(toks)):
        if toks[i] in vals:
            toks[i] = vals[toks[i]]

    # find if there exists !=, >=, <=
    eq_idxs = [idx for idx, tok in enumerate(toks) if tok == "="]
    eq_idxs.reverse()
    prefix = ('!', '>', '<')
    for eq_idx in eq_idxs:
        pre_tok = toks[eq_idx-1]
        if pre_tok in prefix:
            toks = toks[:eq_idx-1] + [pre_tok + "="] + toks[eq_idx+1: ]

    return toks

In [21]:
# sent = "select sum(credit) as T1 from table;"
sent = 'select product_service from ( select product_service, count(distinct strftime("%m", transaction_date)) as n_months from master_txn_table where customers = "[customer_name]" and strftime("%m", transaction_date) >= strftime("%m", current_date) - 6 group by product_service ) where n_months >= 6'

In [22]:
AGG_OPS = ('none', 'max', 'min', 'count', 'sum', 'avg','month','year','week','date')

In [23]:
t = tokenize(sent)

In [26]:

def scan_alias(toks):
    """Scan the index of 'as' and build the map for all alias"""

    as_idxs = [idx for idx, tok in enumerate(toks) if tok == 'as']
    
    alias = {}
    for idx in as_idxs:
        temp = toks[idx-1]

        for i in range(idx,-1,-1):
            if toks[i] in AGG_OPS:
                temp = " ".join(toks[i:idx-1 +1])
                break
        
        alias[toks[idx+1]] = temp
    return alias

In [27]:
scan_alias(t)

{'n_months': 'count ( distinct strftime ( "%m" , transaction_date ) )'}

In [30]:
t.index("select",1)

4